## Loading Data

In [ ]:
import zipfile
with zipfile.ZipFile("/content/novozymes-enzyme-stability-prediction.zip", "r") as zip_ref:
  zip_ref.extractall("Data")

In [1]:
import pandas as pd
train_df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train.csv")

In [2]:
train_df.head()

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


In [3]:
train_updated_df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train_updates_20220929.csv")
train_updated_df.head()

,seq_id,protein_sequence,pH,data_source,tm
0,69,NaN,NaN,NaN,NaN
1,70,NaN,NaN,NaN,NaN
2,71,NaN,NaN,NaN,NaN
3,72,NaN,NaN,NaN,NaN
4,73,NaN,NaN,NaN,NaN


## Updated CSV

In [ ]:
import pandas as pd


df_train = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train.csv", index_col="seq_id")
df_train_updates = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train_updates_20220929.csv", index_col="seq_id")

all_features_nan = df_train_updates.isnull().all("columns")

drop_indices = df_train_updates[all_features_nan].index
df_train = df_train.drop(index=drop_indices)

swap_ph_tm_indices = df_train_updates[~all_features_nan].index
df_train.loc[swap_ph_tm_indices, ["pH", "tm"]] = df_train_updates.loc[swap_ph_tm_indices, ["pH", "tm"]]

In [ ]:
df_train_updates

In [ ]:
df_train

In [2]:
df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train.csv")
updates = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train_updates_20220929.csv")

drop_seq_ids = updates[updates["pH"].isnull()]["seq_id"].values
change_seq_ids = updates[updates["pH"].notnull()]["seq_id"].values

# drop
df.drop(df[df.seq_id.isin(drop_seq_ids)].index, inplace=True)

# change
df.loc[df.seq_id.isin(change_seq_ids), ["pH", "tm"]] = updates[
    updates.seq_id.isin(change_seq_ids)
][["pH", "tm"]].values

df.to_csv("updated_train.csv", index=False)

In [ ]:
df_updated = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Notebooks/updated_train.csv")

In [ ]:
df_updated.head()

## EDA

### BioPandas

In [7]:
import biopandas

In [8]:
somethin = ppdb.df["ATOM"]

NameError: name 'ppdb' is not defined

In [ ]:
wt = somethin.groupby('residue_number').b_factor.agg('first').values

### Continue

In [ ]:
df_test = pd.read_csv("/content/Data/test.csv")
df_test.head()

In [ ]:
df_updated = pd.read_csv("/content/updated_train.csv")
df_updated.drop("data_source", inplace = True, axis = 1)
df_updated.head()

In [ ]:
df_updated.isnull().sum()

In [ ]:
fillVal = df_updated[["pH"]].mean().round()
df_updated.fillna(filblVal, inplace = True)

In [ ]:
df_updated.isnull().sum()

### Encoding the Amino Acid Sequences

#### Try One

##### Hashing Vectorizer

In [ ]:
random_sequence = [df_updated.iloc[1]["protein_sequence"]]

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = HashingVectorizer(n_features = 2**4)

X = vectorizer.fit_transform(random_sequence)

In [ ]:
X

#### TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()

X_ = vec.fit_transform(random_sequence)

In [ ]:
vec.get_feature_names_out()

### Try Two

#### List Strategy (Encoding singular letters)

In [ ]:
import numpy as np

### Engineering Features